In [1]:
import warnings
# 忽视警告
warnings.filterwarnings('ignore')

import cv2
from PIL import Image
import numpy as np
import copy
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
from torch.utils.data import DataLoader

In [2]:
def processing_data(data_path, height=224, width=224, batch_size=64,
                    test_split=0.1):
    """
    数据处理部分
    :param data_path: 数据路径
    :param height:高度
    :param width: 宽度
    :param batch_size: 每次读取图片的数量
    :param test_split: 测试集划分比例
    :return: 
    """
    transforms = T.Compose([
        T.Resize((height, width)),
        T.RandomHorizontalFlip(0.1),  # 进行随机水平翻转
        T.RandomVerticalFlip(0.1),  # 进行随机竖直翻转
        T.ToTensor(),  # 转化为张量
        T.Normalize([0], [1]),  # 归一化
    ])

    dataset = ImageFolder(data_path, transform=transforms)
    # 划分数据集
    train_size = int((1-test_split)*len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    # 创建一个 DataLoader 对象
    train_data_loader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True)
    valid_data_loader = DataLoader(test_dataset, batch_size=batch_size,shuffle=True)

    return train_data_loader, valid_data_loader

In [3]:
torch.cuda.is_available()

True

In [4]:
data_path = './dataset'
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
train_data_loader, valid_data_loader = processing_data(data_path=data_path, height=112, width=112, batch_size=512)
epochs = 200
print('加载完成...')

加载完成...


In [5]:
from model import Arcface
from model import Backbone
class ArcfaceNet(nn.Module):
    def __init__(self, classes=10):
        super(ArcfaceNet, self).__init__()
        self.Resnet = Backbone(num_layers=50,drop_ratio=0.6,mode='ir_se').to(device)
        self.Resnet.load_state_dict(torch.load('model_ir_se50.pth'))
        self.head = Arcface(embedding_size=512, classnum=classes).to(device)

    def forward(self, x):
        out = self.Resnet(x)
        return out

model = ArcfaceNet(classes=10)
optimizer = optim.Adam(model.parameters(), lr=1e-3,weight_decay=5e-4)  # 优化器
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                 'max', 
                                                 factor=0.5,
                                                 patience=2)
criterion = nn.CrossEntropyLoss()  

In [6]:
def train(model,epochs):
    model.train()
    loss_list = []  # 存储损失函数值
    running_loss = 0.    
    for epoch in range(epochs):
        print('epoch {} started'.format(epoch))                            
        for batch_idx,(imgs, labels) in tqdm(enumerate(train_data_loader, 1)):
            imgs = imgs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            embeddings = model(imgs)
            thetas = model.head(embeddings, labels)
            loss = nn.CrossEntropyLoss()(thetas, labels)
            loss.backward()
            running_loss += loss.item()
            optimizer.step  
            loss_list.append(loss)
        print('step:' + str(epoch + 1) + '/' + str(epochs) + ' || Total Loss: %.4f' % (loss))
    torch.save(model.state_dict(), './results/temp.pth')
train(model,epochs)

epoch 0 started


0it [00:00, ?it/s]

step:1/200 || Total Loss: 35.4354
epoch 1 started


0it [00:00, ?it/s]

step:2/200 || Total Loss: 35.2008
epoch 2 started


...

step:198/200 || Total Loss: 35.4577
epoch 198 started


0it [00:00, ?it/s]

step:199/200 || Total Loss: 35.5307
epoch 199 started


0it [00:00, ?it/s]

step:200/200 || Total Loss: 35.5122


In [7]:
train_result = []
for loss in loss_list :
    train_result.append(loss.detach().to('cpu').numpy())
train_result

NameError: name 'loss_list' is not defined

In [ ]:
plt.plot(train_result,label = "loss")
plt.legend()
plt.show()